# Question 1
## c)-d)

In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
class Dynamics:
    pass

In [3]:
hw5dynamics=Dynamics()
hw5dynamics.kappa = 0.472
hw5dynamics.alpha = 4.4
hw5dynamics.sigma = 0.368
hw5dynamics.S0 = 106.9
hw5dynamics.r = 0.05

In [4]:
class Contract:
    pass

In [5]:
hw5contract=Contract()
hw5contract.K1 = 103.2
hw5contract.T1 = 0.5
hw5contract.T2 = 0.75

In [6]:
class MC:
    pass

In [7]:
hw5MC=MC()
hw5MC.N = 100   # Number of timesteps on each path
hw5MC.M = 100000  # Number of paths.  Change this if necessary.
hw5MC.epsilon = 0.01  # For the dC/dS calculation
hw5MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [8]:
def pricer_oilcall_OU_MC(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    
    kappa, alpha, sigma, S0, r = dynamics.kappa, dynamics.alpha, dynamics.sigma, dynamics.S0, dynamics.r
    K1, T1, T2 = contract.K1, contract.T1, contract.T2
    N, M, epsilon, seed = MC.N, MC.M, MC.epsilon, MC.seed
    
    call_price_sim1 = np.zeros(M)
    call_price_sim2 = np.zeros(M)
    
    for i in tqdm(range(M)):
        
        Z = np.random.normal(0,1,N)
        X1 = np.log(S0)
        X2 = np.log(S0+epsilon)
        deltat = T1/N
        
        for j in range(N):
            X1 = X1 + kappa*(alpha - X1)*deltat + sigma*Z[j]*np.sqrt(deltat)
            X2 = X2 + kappa*(alpha - X2)*deltat + sigma*Z[j]*np.sqrt(deltat)
        
        ST1 = np.exp(X1)
        FT1 = np.exp(np.exp(-kappa*(T2-T1))*np.log(ST1) + (1-np.exp(-kappa*(T2-T1)))*alpha + sigma**2/(4*kappa)*(1-np.exp(-2*kappa*(T2-T1))))
        call_price_sim1[i] = np.exp(-r*T1)*np.maximum(FT1-K1,0)
        
        ST2 = np.exp(X2)
        FT2 = np.exp(np.exp(-kappa*(T2-T1))*np.log(ST2) + (1-np.exp(-kappa*(T2-T1)))*alpha + sigma**2/(4*kappa)*(1-np.exp(-2*kappa*(T2-T1))))
        call_price_sim2[i] = np.exp(-r*T1)*np.maximum(FT2-K1,0)
        
    
    call_price = np.mean(call_price_sim1)
    call_price2 = np.mean(call_price_sim2)
    standard_error = np.std(call_price_sim1)/np.sqrt(M)      
    call_delta = (call_price2 - call_price)/epsilon   
        
        
        
    return(call_price, standard_error, call_delta)
        

In [9]:
(call_price, standard_error, call_delta) = pricer_oilcall_OU_MC(hw5contract,hw5dynamics,hw5MC)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:57<00:00, 1737.63it/s]


In [10]:
print("Call Price: {:.4f} \nStandard Error: {:.4f} \nDelta: {:.4f}".format(call_price, standard_error, call_delta))

Call Price: 7.8214 
Standard Error: 0.0419 
Delta: 0.3444


# Question 2
## a)

In [11]:
from scipy import stats

# S is the (spot) price of the underlying
# Assumes zero dividends

def BS(sigma,S,r,K,T):
    F = S*np.exp(r*T)
    sd = sigma*np.sqrt(T)
    if sd==0:
        return np.maximum(np.exp(-r*T)*(F-K),0)
    d1 = np.log(F/K)/sd+sd/2
    d2 = np.log(F/K)/sd-sd/2
    return np.exp(-r*T)*(F*stats.norm.cdf(d1)-K*stats.norm.cdf(d2))

def BSdelta(sigma,S,r,K,T):
    F = S*np.exp(r*T)
    sd = sigma*np.sqrt(T)
    if sd==0:
        return (1 if F>K else 0)
    d1 = np.log(F/K)/sd+sd/2
    return stats.norm.cdf(d1)


In [12]:
hw5p2dynamics=Dynamics()
hw5p2dynamics.S0 = 100
hw5p2dynamics.r = 0
hw5p2dynamics.impliedsigma = 0.30
hw5p2dynamics.mu = 0.05
hw5p2dynamics.realizedsigma = 0.40

In [13]:
hw5p2contract=Contract()
hw5p2contract.K = 100 
hw5p2contract.T = 25/252    #option has 25 trading days to expiry

In [14]:
class HedgingStrategy:
    pass

In [15]:
hw5p2strategy = HedgingStrategy()
hw5p2strategy.dt = 1/252
hw5p2strategy.numRebalances = 20   #This is called "N" in the homework assignment.  Running trading strategy for 20 days. 

#Time 0 is when we enter the trading strategy
#Time (numRebalances*dt) is when we exit the trading strategy
#Therefore numRebalances and dt should be chosen such that numRebalances*dt <= option expiry
#If the inequality is strict (numRebalances*dt < option expiry), then that simply
#means that the date when you exit out of the strategy (closing all option and stock positions) 
#is a date when the option has not yet expired.


In [16]:
hw5p2MC=MC()
hw5p2MC.M = 10000              # Number of paths  
hw5p2MC.dt = hw5p2strategy.dt # time step  
hw5p2MC.seed = 0              # Seeding the random number generator with a specified number helps make the calculations reproducible

In [18]:
results = simulate_deltahedgedCall_GBM(hw5p2contract,hw5p2dynamics,hw5p2strategy,hw5p2MC)
np.mean(results)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:25<00:00, 389.00it/s]


1.0246923995724533

In [19]:
hw5p2dynamics.realizedsigma = 0.20
results = simulate_deltahedgedCall_GBM(hw5p2contract,hw5p2dynamics,hw5p2strategy,hw5p2MC)
np.mean(results)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:25<00:00, 385.73it/s]


-0.9660449123021956

## b)

In [20]:
hw5p2dynamics.mu = -0.05
hw5p2dynamics.realizedsigma = 0.40
results = simulate_deltahedgedCall_GBM(hw5p2contract,hw5p2dynamics,hw5p2strategy,hw5p2MC)
np.mean(results)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:28<00:00, 348.04it/s]


1.0221510308217534

In [21]:
hw5p2dynamics.realizedsigma = 0.20
results = simulate_deltahedgedCall_GBM(hw5p2contract,hw5p2dynamics,hw5p2strategy,hw5p2MC)
np.mean(results)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:26<00:00, 383.35it/s]


-0.9644590679835167